<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_04_YogFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the LICENSE file in the root directory of this source tree.

#Filtering out charts with specific yogs


*   Budhaditya_Yog
*   Gajakesari_Yog
*   Raj_Yog
*   Chandra Mangal Yog
*   Dhan Yog



For those who may not have access to a live MongoDB instance, we have set up a mechanism where the JSON file that would have been loaded into a hosted MongoDB instance, will be loaded into a locally (that is in the VM) installed MongoDB instance and the queries can be run in a manner that is identical to a live, hosted MongoDB instance

#SetUp Environment

##Install Software

In [1]:
!pip -qq install python-docx                #https://python-docx.readthedocs.io/en/latest/
!/usr/bin/python3 -m pip install "pymongo[srv]"
!apt install mongodb > /dev/null            # install local MongoDB server           
#!service mongodb start                      # start Local MongoDB server instance


     |████████████████████████████████| 5.6 MB 6.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 7.5 MB/s 




##Install, import prerequisites

In [2]:
#Utility functions 
from pymongo import MongoClient
import pandas as pd
import dateutil
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from datetime import timedelta
import pytz

#import matplotlib.pyplot as plt
#import math


In [38]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py      # retrieval related utilities
!wget -O p21YogInfo.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21YogInfo.py    # yog data 
# ---------------------------

!wget -O Saraswati.png -q https://github.com/prithwis/parashar21/raw/main/images/Saraswati02.png                 
!wget -O p21logo.png -q https://github.com/prithwis/parashar21/raw/main/images/p21logo-002-1.png                  

In [55]:
#!cat p21utils.py
import p21
import p21utils
import p21utilsR
import p21YogInfo
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21utils)
importlib.reload(p21utilsR)
importlib.reload(p21YogInfo)

p21utils imported
p21utilsR imported


<module 'p21YogInfo' from '/content/p21YogInfo.py'>

In [ ]:
#!cat p21YogInfo.py

#Local or Remote Atlas

##Local Database Operations

In [ ]:
!service mongodb start                      # start Local MongoDB server instance

In [ ]:
mongoURI = "mongodb://localhost:27017/"                               # for remote server
client = MongoClient(mongoURI)

db = client.parashar21Local
#pd.DataFrame(list(db.list_collections()))

###Download Horoscope data in JSON and load into MongoDB

In [ ]:
# Retrieve JSON file from Github and load in local Mongo
#!wget -O peopleData.json -q https://github.com/prithwis/parashar21/raw/main/data/peopleDataTest5.json

#!gdown -O peopleData.json https://drive.google.com/uc?id=1_VBHE7Hyue8t3hdBdLU6lgpnaWhhRVd4  # 39662 documents OLD file
!gdown  -O peopleData.json https://drive.google.com/uc?id=1z4JFd-zCaS71dICKIYx46kQGbXPAHq5C  # 39662 documents
!mongoimport --db parashar21Local --collection khona21Local --drop --file peopleData.json

Downloading...
From: https://drive.google.com/uc?id=1z4JFd-zCaS71dICKIYx46kQGbXPAHq5C
To: /content/peopleData.json
100% 134M/134M [00:01<00:00, 69.4MB/s]
2022-09-18T12:20:49.024+0000	connected to: localhost
2022-09-18T12:20:49.024+0000	dropping: parashar21Local.khona21Local
2022-09-18T12:20:52.025+0000	[#####...................] parashar21Local.khona21Local	27.4MB/128MB (21.4%)
2022-09-18T12:20:55.023+0000	[##########..............] parashar21Local.khona21Local	55.0MB/128MB (42.9%)
2022-09-18T12:20:58.026+0000	[###############.........] parashar21Local.khona21Local	83.4MB/128MB (65.1%)
2022-09-18T12:21:01.030+0000	[####################....] parashar21Local.khona21Local	111MB/128MB (86.6%)
2022-09-18T12:21:02.920+0000	[########################] parashar21Local.khona21Local	128MB/128MB (100.0%)
2022-09-18T12:21:02.920+0000	imported 39662 documents


In [ ]:
kollection = db.khona21Local                   # Set the collection to work with
#kollection = db.khona21                   # Set the collection to work with
totalCharts = kollection.count_documents({})
dbSize = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
print(totalCharts, ' documents,',round(dbSize,2), " KB data")

39662  documents, 142321.85  KB data


##Remote Atlas Database

In [5]:
from google.colab import drive
drive.mount('/content/drive')
# -------------------------------------------
# copy credentials file
!cp /content/drive/'My Drive'/Praxis/WebCredentials/Parashar21_Atlas_18Sep22.py credentials.py
# -------------------------------------------
# MongoDB credentials are stored here
#
from credentials import parashar21URI
# -------------------------------------------
mongoURI = parashar21URI                                   # for remote ATlas server

client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']
# ----------------------------------------------

Mounted at /content/drive


['Parashar21', 'admin', 'local']

In [6]:
# Show existing Collections
db = client.Parashar21
pd.DataFrame(list(db.list_collections()))

,name,type,options,info,idIndex
0,khona21,collection,{},"{'readOnly': False, 'uuid': b'Ia\xff*\xf3\xbfF...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_'}"


In [7]:
#kollection = db.khona21Test                   # Set the collection to work with
kollection = db.khona21                   # Set the collection to work with
totalCharts = kollection.count_documents({})
print(totalCharts)

39662


#Report Generation Function
Please note that the generated report MS Office doc format **will be stored in the Colab VM drive** and needs to be downloaded to local machine for viewing

In [49]:
def genYogReport(yText, ChartStyle,Sample = True):
    
    if Sample:
        df4Report = df.sample(5)
    else:
        df4Report = df
    
    selectedCharts = len(df)
    printCharts = len(df4Report)
    #printStatus = str(printCharts)+' charts of total '+str(totalCharts)+' printed.'
    printStatus = str(len(df))+' charts of total '+str(totalCharts)+' selected and '+str(printCharts)+' printed'
    
    

    now = datetime.now(pytz.timezone('Asia/Kolkata'))
    fileName = 'p21Yog_'+yText[0:4]+'_'+ChartStyle[0:1]+'_'+now.strftime("%H%M%S")+'.doc'
    curQueryStr = json.dumps(selCond)[0:100]
    # ---------------------------------------------------------
    p21utilsR.R01_CreateReportDoc(yText,printStatus)                      # Creates MS Word Doc called p21.document
    # ----------------------------------------------------------
    for chart in range(len(df4Report)):
        chartData =df4Report.iloc[chart]
        #R51_processChart(chartData)
        p21utilsR.R511_parseChartData(chartData)              # Break up database data into useful pieces
                                            # Creates two primary dicts, GLon, GRet
        for ctype in ['Rashi','Navamsa']:
        #for ctype in ['Rashi']:
            p21.ChartType = ctype
            p21utils.R11_LocateGrahaInRashi()   # Converts Graha Long to Rashi positions
                                                # Rashi Num as well as Rashi name
            if ChartStyle == 'Bengal':
                p21utilsR.R12B_drawChart_Bengal()    # Draw Chart in Bengal style
            if ChartStyle == 'South':
                p21utilsR.R12B_drawChart_South()    # Draw Chart in South style
            if ChartStyle == 'North':
                p21utilsR.R12B_drawChart_North()    # Draw Chart in North style
            #
            
                                                
        #print(p21.GrahaBhava)
        #print(p21.LordBhav)

        p21.yogsFound = []
        for yogName in list(p21YogInfo.yogCond.keys()):                                             # Yog conditions picked up from p21YogInfo.py
            objList = list(kollection.find(p21YogInfo.yogCond[yogName],{"pid.name" : 1,"_id":0}))   # Get names of people whose charts have these Yogs
            #print(rawList)
            personList = list(map(lambda e:e['pid']['name'],objList))                               # Convert list of json object to list of string
            #print(personList)
            if p21.pName in personList:
                p21.yogsFound.append(yogName)
                #print("found ", yogName)

        p21utilsR.R512_FormatPage()
        #print('----')
    # ------------------------------------
    p21.document.save(fileName)
    


#Retrieve Charts 
Based on selective queries <br>
All output messages suppressed <br>
Reports will be available as DOC files in the VM drive <br>
Please specify style of report, 'Bengal', 'North' or 'South' while calling the report generation routine <br>
genSampleReport('Bengal',Sample = False), specifying Sample='False' will stop sampling and print reports for all charts retrieved. Otherwise, by default, only 5 charts, chosen at random will be printed in the DOC file <br>
genSampleReport() replaced by genYogReport() with two additional parameters, namely YogName and YogDecription that are printed in the report


##Moon Based Yog

In [56]:
%%capture
yogName = 'Kemadruma'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

In [50]:
%%capture
yogName = 'Dhurdhura'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

In [30]:
%%capture
yogName = 'Anapha'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

In [26]:
%%capture
yogName = 'Sunapha'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

## Gajakesari Yog

In [17]:
%%capture
#from p21YogInfo import  yogCond, yogText
yogName = 'Gajakesari'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
genYogReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
print(p21.GRashiN)

{'La': 9, 'Su': 11, 'Mo': 7, 'Ma': 4, 'Me': 1, 'Ju': 1, 'Ve': 3, 'Sa': 3, 'Ra': 4, 'Ke': 10}


## zBudhaditya Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'BudhAditya'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genYogReport(yogText[yogName],'Bengal')

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'BudhAdityaNR'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genYogReport(yogText[yogName],'Bengal')

##zRaj Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'Rajyog'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genYogReport(yogText[yogName],'Bengal')

##Chandra Mangal (Mahalakshmi) Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'ChandraMangal2'
#yogName = 'ChandraMangal1' # this returns no records in our database
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genYogReport(yogText[yogName],'Bengal')

##Dhan Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'DhanYog'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genYogReport(yogText[yogName],'Bengal')

##All Yogs

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
#yogList = ['BudhAditya','BudhAdityaNR', 'Gajakesari', 'Rajyog', 'ChandraMangal2','DhanYog']
for yogName in list(p21YogInfo.yogCond.keys()):
    #selCond = yogCond[yogName]
    df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
    #print(yogName, len(df))
    if len(df) > 0:
        genYogReport(yogText[yogName],'Bengal')


#Tests

In [ ]:
%%capture
yogText = 'Dhan^ yog : Lords 1 with 2 ^ ref : LHAE KN Rao, pg 77'
selCond = { 'BLConjunctsBL2.1': {'$in': [2,3]}        }

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genYogReport(yogText,'Bengal')

In [ ]:
%%capture
yogText = 'Dummy^ yog : Ma not conjuct with Moon ^ ref : https://www.astroyogi.com/kundli/yog/mahalaxmi'
selCond = {"$and":
                    [
                        {'GConjunctsG2.Ma' : {'$in': ['Su']}} ,     # Ma not conjunct Mo 
                        {'GConjunctsG2.Ma' : {'$nin': ['Mo']}} ,     # Ma not conjunct Mo        
                        {'GrahaBhava.Mo' : {'$nin': [1,2,3,4,5,6,7,8,9,10]}}     # Mo not in 3,6,8,12
                        
                    ]
         }

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genYogReport(yogText,'Bengal')

In [ ]:
%%capture
yogText = 'Dummy^ yog : Lords 1 with 2,3 ^ ref : LHAE KN Rao, pg 77'
selCond = { 'BLConjunctsBL2.1': {'$in': ['2','3']} }

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genYogReport(yogText,'Bengal')

In [46]:
!rm *.doc